# Treinamento

## KNN

### KNN - Train Test Split

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
############## SEPARANDO ENTRE DADOS DE TREINO E TESTE ##############

# Pega as duas primeiras colunas (key, nome_arqivo) do arquivo LBP.csv
df = pd.read_csv('./data/caracteristicas/LBP.csv', usecols=[0,1])

# Remove as linhas onde há um nome_arquivo repetido
df = df.drop_duplicates(subset=['nome_arquivo'])

# Remova o elemento em que só há uma ocorrência de classe
df = df.groupby('key').filter(lambda x: len(x) > 1)

# Coloca a coluna key em uma lista chamada classes e a coluna nome_arquivo em uma lista chamada arquivos
classes = df['key'].tolist()
arquivos = df['nome_arquivo'].tolist()

# Separe quais arquivos serão usados para treino e quais serão usados para teste
arquivos_treino, arquivos_teste, classes_treino, classes_teste = train_test_split(arquivos, classes, test_size=0.2, random_state=42, stratify=classes)

# Carregar o arquivo LBP.csv
df_lbp = pd.read_csv('./data/caracteristicas/LBP.csv')

# Selecionar apenas as linhas que estão nos arquivos de treino
df_lbp_treino = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_treino)]

# Selecionar apenas as linhas que estão nos arquivos de teste
df_lbp_teste = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_teste)]


# Separa entre y_treino e X_treino
y_treino = df_lbp_treino['key'].values
X_treino = df_lbp_treino.drop(['key', 'nome_arquivo'], axis=1).values

# Separa entre y_teste e X_teste
y_teste = df_lbp_teste['key'].values
X_teste = df_lbp_teste.drop(['key', 'nome_arquivo'], axis=1).values

In [ ]:
############## NORMALIZANDO OS DADOS ##############

# Cria um objeto para normalizar os dados
scaler = StandardScaler()

# Normaliza os dados de treino
X_treino = scaler.fit_transform(X_treino)

# Normaliza os dados de teste
X_teste = scaler.transform(X_teste)

In [ ]:
############## TRATANDO OS VALORES NAN ##############

# Substitui os valores NaN por 0
X_treino = np.nan_to_num(X_treino)
X_teste = np.nan_to_num(X_teste)

In [ ]:
############## ENCONTRA OS MELHORES PARAMETROS KNN ##############

# Cria um objeto do tipo KNN
knn = KNeighborsClassifier()

# Define os valores que serão testados para cada parâmetro
k_range = list(range(1, 20, 2))

# Cria um dicionário com os valores dos parâmetros
parametros_knn = dict(n_neighbors=k_range)

# Cria um objeto do tipo StratifiedKFold
skf = StratifiedKFold(n_splits=5)

# Cria um objeto do tipo GridSearchCV
grid_knn = GridSearchCV(knn, parametros_knn, cv=skf, scoring='accuracy', verbose=1, n_jobs=-1)

# Treina o modelo
grid_knn.fit(X_treino, y_treino)

# Imprime os melhores parâmetros
print(f'Melhor valor para n_neighbors: {grid_knn.best_params_["n_neighbors"]}')
print(f'Melhor Acurácia: {grid_knn.best_score_}')

############## TREINANDO O MODELO KNN COM OS MELHORES PARAMETROS ##############

# Cria um objeto do tipo KNN
knn = KNeighborsClassifier(n_neighbors=grid_knn.best_params_["n_neighbors"])

# Treina o modelo
knn.fit(X_treino, y_treino)

# Testa o modelo
y_pred = knn.predict(X_teste)


In [ ]:
############## VOTAÇÃO ##############

# copie o df df_lbp_teste para df_lbp_teste_votacao e adicione a coluna 'predicao'
df_lbp_teste_votacao = df_lbp_teste.copy()
df_lbp_teste_votacao['predicao'] = y_pred

# Faça a votação para cada nome_arquivo
df_lbp_teste_votacao = df_lbp_teste_votacao.groupby('nome_arquivo')['predicao'].agg(lambda x:x.value_counts().index[0]).reset_index()

# Crie um df com as classes reais
df_lbp_teste_real = df_lbp_teste.drop_duplicates(subset=['nome_arquivo'])

# Junte os dois df
df_lbp_teste_real = df_lbp_teste_real.merge(df_lbp_teste_votacao, on='nome_arquivo')

In [ ]:
############## AVALIANDO O MODELO ##############

# Imprime o relatório de classificação
print(classification_report(df_lbp_teste_real['key'], df_lbp_teste_real['predicao']))


### KNN - SKFold

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

############## SEPARANDO ENTRE DADOS DE TREINO E TESTE ##############

# Pega as duas primeiras colunas (key, nome_arqivo) do arquivo LBP.csv
df = pd.read_csv('../data/caracteristicas/LBP.csv', usecols=[0,1])

# Remove as linhas onde há um nome_arquivo repetido
df = df.drop_duplicates(subset=['nome_arquivo'])

# Remova o elemento em que só há uma ocorrência de classe
df = df.groupby('key').filter(lambda x: len(x) > 1)

# Coloca a coluna key em uma lista chamada classes e a coluna nome_arquivo em uma lista chamada arquivos
classes = df['key'].tolist()
arquivos = df['nome_arquivo'].tolist()

# Cria o objeto skf que divide os dados em 10 partes
skf = StratifiedKFold(n_splits=10)
acc = []
f1s = []

# Divide os dados em 10 partes
for train_index, test_index in skf.split(arquivos, classes):
    # Separe quais arquivos serão usados para treino e quais serão usados para teste
    arquivos_treino, arquivos_teste = np.array(arquivos)[train_index], np.array(arquivos)[test_index]
    classes_treino, classes_teste = np.array(classes)[train_index], np.array(classes)[test_index]

    print(arquivos_treino)

    # Carregar o arquivo LBP.csv
    df_lbp = pd.read_csv('../data/caracteristicas/LBP.csv')

    # Selecionar apenas as linhas que estão nos arquivos de treino
    df_lbp_treino = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_treino)]

    # Selecionar apenas as linhas que estão nos arquivos de teste
    df_lbp_teste = df_lbp[df_lbp['nome_arquivo'].isin(arquivos_teste)]

    # Separa entre y_treino e x_treino
    y_treino = df_lbp_treino['key']
    x_treino = df_lbp_treino.drop(['key', 'nome_arquivo'], axis=1)

    # Separa entre y_teste e x_teste
    y_teste = df_lbp_teste['key']
    x_teste = df_lbp_teste.drop(['key', 'nome_arquivo'], axis=1)

    # Normaliza os dados
    scaler = StandardScaler()
    scaler.fit(x_treino)
    x_treino = scaler.transform(x_treino)
    x_teste = scaler.transform(x_teste)

    # tratando so valores NaN
    x_treino = np.nan_to_num(x_treino)
    x_teste = np.nan_to_num(x_teste)

    # Encontra os melhores parâmetros para o KNN
    knn = KNeighborsClassifier()
    k_range = list(range(1, 20, 2))
    parametros_knn = dict(n_neighbors=k_range)
    skf = StratifiedKFold(n_splits=5)
    grid_knn = GridSearchCV(knn, parametros_knn, cv=skf, scoring='accuracy', verbose=1, n_jobs=-1)

    grid_knn.fit(x_treino, y_treino)

    print(f'Melhor valor de K: {grid_knn.best_params_["n_neighbors"]}')
    print(f'Melhor valor de acurácia: {grid_knn.best_score_}')

    # Treina o KNN com os melhores parâmetros
    knn = KNeighborsClassifier(n_neighbors=grid_knn.best_params_["n_neighbors"])
    knn.fit(x_treino, y_treino)

    # Faz a predição
    y_pred = knn.predict(x_teste)

    # Votação
    df_lbp_teste_votacao = df_lbp_teste.copy()
    df_lbp_teste_votacao['predicao'] = y_pred
    df_lbp_teste_votacao = df_lbp_teste_votacao.groupby('nome_arquivo')['predicao'].agg(lambda x:x.value_counts().index[0]).reset_index()

    df_lbp_teste_real = df_lbp_teste.drop_duplicates(subset=['nome_arquivo'])
    df_lbp_teste_real = df_lbp_teste_real.merge(df_lbp_teste_votacao, on='nome_arquivo')

    # Calcula a acurácia
    acc.append(accuracy_score(df_lbp_teste_real['key'], df_lbp_teste_real['predicao']))

    # Calcula a f1-score
    f1s.append(classification_report(df_lbp_teste_real['key'], df_lbp_teste_real['predicao'], output_dict=True)['weighted avg']['f1-score'])

print(f'Acurácia: {np.mean(acc)}')
print(f'F1-Score: {np.mean(f1s)}')
    


['XC208241' 'XC210730' 'XC211242' 'XC225377' 'XC234296' 'XC239627'
 'XC247566' 'XC248033' 'XC248034' 'XC252931' 'XC253252' 'XC253349'
 'XC253722' 'XC253981' 'XC254203' 'XC256096' 'XC256217' 'XC256529'
 'XC263623' 'XC267418' 'XC267430' 'XC267431' 'XC267433' 'XC267653'
 'XC267929' 'XC268755' 'XC268804' 'XC269815' 'XC270309' 'XC278555'
 'XC282921' 'XC284630' 'XC288339' 'XC288343' 'XC289357' 'XC289358'
 'XC303791' 'XC304029' 'XC306046' 'XC306077' 'XC306581' 'XC309194'
 'XC309534' 'XC312064' 'XC317542' 'XC317544' 'XC318261' 'XC319599'
 'XC320886' 'XC324889' 'XC324914' 'XC327328' 'XC327364' 'XC329025'
 'XC330211' 'XC330429' 'XC331493' 'XC333650' 'XC334130' 'XC335495'
 'XC337180' 'XC337979' 'XC337983' 'XC337985' 'XC339819' 'XC339821'
 'XC341167' 'XC343867' 'XC344095' 'XC344096' 'XC349298' 'XC349311'
 'XC350350' 'XC356315' 'XC361216' 'XC361217' 'XC361218' 'XC361219'
 'XC361221' 'XC361222' 'XC361223' 'XC361224' 'XC361226' 'XC361228'
 'XC361230' 'XC361232' 'XC361235' 'XC361236' 'XC361239' 'XC361

## SVM